In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
from anndata import AnnData
import os
import logging
import json

In [ ]:
# load chrom info
gtf_file = '/home/share/huadjyin/home/linadi/wqr_files/Projs/20250123_Multiomics3/multiminer/references/Homo_sapiens.GRCh38.113.gff3'
gtf = pd.read_csv(gtf_file, sep='\t', comment='#', header=None)
genes = gtf[gtf[2].isin(['gene', 'ncRNA_gene', 'pseudogene'])]

# gene info
gene_df = genes[[0, 3, 4, 6, 8]].copy()
gene_df.columns = ['chrom', 'start', 'end', 'strand', 'info']

gene_df['gene_id'] = gene_df['info'].str.extract(r'gene_id=([^;]+)')
if gene_df['gene_id'].isna().all():
    print("Failed to extract gene_id from GTF file.")

# 计算 TSS 位置
gene_df['tss'] = np.where(gene_df['strand'] == '+', gene_df['start'], gene_df['end'])
gene_df['chrom'] = gene_df['chrom'].astype(str)

# chrom length info
data = []
with open(gtf_file, "r") as f:
    for line in f:
        if line.startswith("##sequence-region"):
            tokens = line.strip().split()
            # tokens 格式: ["##sequence-region", chrom, start, end]
            if len(tokens) >= 4:
                chrom = tokens[1]
                length = int(tokens[3])
                data.append({'chrom': chrom, 'length': length})

# 转换为 DataFrame
chrom_info_df = pd.DataFrame(data)
allowed_chroms = [str(i) for i in range(1, 23)] + ['X', 'Y']

# 过滤 DataFrame，只保留 chrom 列中值在 allowed_chroms 中的行
chrom_info_df = chrom_info_df[chrom_info_df['chrom'].isin(allowed_chroms)]
chrom_info_dict = chrom_info_df.set_index('chrom').to_dict()['length']
chrom_info_dict = {str(k): v for k, v in chrom_info_dict.items()}

chrom_info_dict

In [ ]:
def compute_global_position(gene_df, chrom_info_dict):
    """
    compute gene global position。
    
    参数：
      gene_df: 包含至少 'chrom' 和 'tss' 列的 DataFrame
      chrom_info_dict: 字典，键为染色体名称（字符串），值为该染色体的长度（整数）
      
    返回：
      修改后的 gene_df，新增一列 'position'
    """
    # 定义染色体的顺序
    ordered_chroms = [ str(i) for i in range(1, 23)] + ['X', 'Y']
    
    # 构建一个字典，将每个染色体映射为前置染色体的总长度
    prefix_length = {}
    running_sum = 0
    for chrom in ordered_chroms:
        prefix_length[chrom] = running_sum
        if chrom in chrom_info_dict:
            running_sum += chrom_info_dict[chrom]
    

    gene_df['position'] = gene_df.apply(lambda row: prefix_length.get(row['chrom'], 0) + row['tss'], axis=1)
    
    return gene_df, prefix_length

gene_df, chrom_prefix_length = compute_global_position(gene_df, chrom_info_dict)
gene_position_df = gene_df.loc[:, ['chrom', 'gene_id', 'tss', 'position']]
gene_position_dict = dict(zip(gene_position_df["gene_id"], gene_position_df["position"]))




In [ ]:

with open("chrom_prefix_length.json", "w") as json_file:
    json.dump(chrom_prefix_length, json_file)
with open("gene_global_position_dict.json", "w") as json_file:
    json.dump(gene_position_dict, json_file)